Generation of image embeddings using EVA_CLIP

In [ ]:
# Colab
from google.colab import drive
drive.mount("/content/drive/")
drive_path="/content/drive/Mydrive"
import condacolab
condacolab.install()

Mounted at /content/drive/


In [ ]:
!git clone https://github.com/baaivision/EVA
!cd EVA/EVA-CLIP

In [ ]:
!pip install torch==2.1.0
!pip install torchaudio 2.1.0+cu118
!pip install torchdata 0.7.0
!pip install torchtext 0.16.0
!pip install -q condacolab
!pip install pillow
!conda create --name rei python=3.8 -y
!conda activate rei
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install -r requirements.txt
!pip install urlextract
!pip install timm
!pip install einops
!pip install apex
!pip install xformers

ERROR: Invalid requirement: '2.1.0+cu118'
ERROR: Could not find a version that satisfies the requirement 0.7.0 (from versions: none)
ERROR: No matching distribution found for 0.7.0
ERROR: Could not find a version that satisfies the requirement 0.16.0 (from versions: none)
ERROR: No matching distribution found for 0.16.0


In [ ]:
!cd /content/EVA/EVA-CLIP/rei

In [ ]:

import pandas as pd
import numpy as np
from urlextract import URLExtract
import urllib.request
from PIL import Image

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/behaviour_simulation_train.xlsx")
df["media"][0]

In [ ]:
extractor = URLExtract()
example_text = df["media"][12]

for url in extractor.gen_urls(example_text):
    print(url)

https://pbs.twimg.com/media/ESCO6b9WkAUBI1o.jpg
https://video.twimg.com/amplify_video/1234143435654868992/pl/nYicdFU2bqnqhT7Q.m3u8?tag=13&v=f4c
https://video.twimg.com/amplify_video/1234143435654868992/vid/640x360/6x9561bKICzF89Ur.mp4?tag=13
https://video.twimg.com/amplify_video/1234143435654868992/vid/480x270/4gUTJJGGySp_lN0e.mp4?tag=13
https://video.twimg.com/amplify_video/1234143435654868992/vid/960x540/0lhTA1u0RQ7IYC-O.mp4?tag=13


In [ ]:
url

'https://pbs.twimg.com/media/D2qiPBTX0AI_13F?format=jpg&name=large'

In [ ]:
urllib.request.urlretrieve(url,"img.png")

img = Image.open("img.png")
img.show()

In [ ]:
import torch
from eva_clip import create_model_and_transforms, get_tokenizer
from PIL import Image

model_name = "EVA02-CLIP-B-16"
pretrained = "eva_clip" # or "/path/to/EVA02_CLIP_B_psz16_s8B.pt"

image_path = "/content/img.png"
caption = ["a food", "a dog", "a cat"]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = create_model_and_transforms(model_name, pretrained, force_custom_clip=True)
tokenizer = get_tokenizer(model_name)
model = model.to(device)

image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

text = tokenizer(["a food", "a dog", "a cat"]).to(device)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True) # Normalization

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[0.8275, 0.1372, 0.0352]]

/usr/local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


Please 'pip install apex'


EVA02_CLIP_B_psz16_s8B.pt:   0%|          | 0.00/300M [00:00<?, ?B/s]

Label probs: tensor([[0.9969, 0.0011, 0.0020]], device='cuda:0')


In [ ]:
image_features

tensor([[ 0.0219, -0.0235, -0.0206, -0.0136,  0.1071,  0.0174,  0.0493,  0.0485,
          0.0907, -0.0531,  0.0611,  0.0215, -0.0175,  0.0111,  0.0742,  0.0364,
          0.0588,  0.0356, -0.0110, -0.0175,  0.0095,  0.0244,  0.0454,  0.0513,
         -0.0114, -0.0167,  0.0517, -0.0212, -0.0616, -0.0071,  0.0261, -0.0207,
         -0.0814,  0.0308,  0.0374, -0.0246,  0.0500,  0.0323, -0.0030, -0.0542,
          0.0341,  0.0211, -0.0520,  0.0593, -0.0605, -0.0136, -0.0616, -0.0264,
          0.0691,  0.0442,  0.0426,  0.0395, -0.0025,  0.0980, -0.0202,  0.0007,
          0.0025,  0.0492, -0.0036,  0.0653,  0.0289,  0.0162, -0.0288,  0.0117,
         -0.0333,  0.0357,  0.0522,  0.0784,  0.0131,  0.0220,  0.0578,  0.0116,
         -0.0070, -0.0199,  0.0144, -0.0125,  0.0396,  0.0475,  0.0053,  0.0003,
          0.0624, -0.0714, -0.0443, -0.0393, -0.0298, -0.0326,  0.0377, -0.0798,
          0.0395,  0.0823,  0.0253, -0.0469,  0.0319, -0.0133, -0.0134,  0.0147,
          0.0379,  0.0055,  